In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install google-adk

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

In [ ]:
import os
import random
import time
import datetime # Necessary for the get_current_date() function
import vertexai
from kaggle_secrets import UserSecretsClient
from vertexai import agent_engines
from google.genai import types

from google.adk.agents import Agent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.runners import InMemoryRunner

print("✅ ADK components imported successfully.")

In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [ ]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

In [ ]:
import datetime

def get_current_date() -> str:
    """
    Returns the current date and time in UTC, which is essential for 
    the LLM to accurately calculate relative dates (e.g., 'next Monday').
    """
    now = datetime.datetime.now(datetime.timezone.utc)
    
    # Example output: "The current date is Monday, December 1, 2025, at 10:35 AM UTC."
    return now.strftime("The current date is %A, %B %d, %Y, at %I:%M %p UTC.")

In [ ]:
# --- Global State for the To-Do List ---
_TASKS = {}
_NEXT_ID = 1

def add_task(description: str) -> str:
    """Adds a new task to the list. Use this function when the user says 'add', 'create', or 'put on the list'."""
    global _NEXT_ID
    # Update global state
    task_id = _NEXT_ID
    _TASKS[task_id] = {"description": description, "completed": False}
    _NEXT_ID += 1
    return f"Task '{description}' successfully added with ID {task_id}."

def view_list() -> str:
    """Retrieves and formats the current to-do list. Use this when the user asks 'what's on my list' or 'show list'."""
    if not _TASKS:
        return "Your to-do list is currently empty."
    
    output = ["✅ Your Current To-Do List:"]
    sorted_tasks = sorted(_TASKS.items()) 
    
    for task_id, task in sorted_tasks:
        status = "[x]" if task['completed'] else "[ ]"
        output.append(f"  {status} (ID {task_id}): {task['description']}")
        
    return "\n".join(output)

def mark_complete(task_id: int) -> str:
    """Marks a task as complete using its ID."""
    if task_id in _TASKS:
        _TASKS[task_id]['completed'] = True
        return f"Task ID {task_id} ('{_TASKS[task_id]['description']}') marked as complete."
    return f"Error: Task with ID {task_id} not found."
def delete_task(task_id: int) -> str:
    """
    Removes a specific task from the list entirely using its ID.
    """
    global _TASKS
    
    if task_id not in _TASKS:
        return f"Error: Task with ID {task_id} not found in the list."
    
    description = _TASKS[task_id]['description']
    del _TASKS[task_id]
    return f"Task ID {task_id} ('{description}') has been permanently removed from the list."
print("✅ Core To-Do List functions defined.")

In [ ]:
import json
from typing import Dict, Any



def orchestrate_task() -> str:
    """
    Analyzes the current to-do list, prioritizes the next task, and determines 
    if it requires delegation to a specialist agent or is a manual task.
    
    Returns:
        A JSON string detailing the next pending task: 
        {"id": 1, "task_description": "Book a flight to Paris", "requires_agent": true}
    """
    global _TASKS
    
    # Prioritize: Find the first incomplete task based on ascending ID (oldest task)
    sorted_tasks = sorted(_TASKS.items())
    
    # First pass: find the next agent-assisted task
    for task_id, task in sorted_tasks:
        if not task['completed']:
            description = task['description'].lower()
            requires_agent = (
                'buy' in description or 'order' in description or 'shopping' in description or
                'book' in description or 'flight' in description or 'train' in description or
                'hotel' in description or 'stay' in description or 'movie' in description or
                'ticket' in description or 'showtime' in description or 'buy' in description or
                'order' in description or 'shopping' in description or 'study' in description or
                'quiz' in description or 'guide' in description or 'plan' in description
            )
            
            if requires_agent:
                 next_task: Dict[str, Any] = {
                    "id": task_id,
                    "task_description": task['description'],
                    "requires_agent": True
                }
                 return json.dumps(next_task)
            
    # Second pass: if no agent tasks, find the next manual task (these are handled last)
    for task_id, task in sorted_tasks:
        if not task['completed']:
            next_task: Dict[str, Any] = {
                "id": task_id,
                "task_description": task['description'],
                "requires_agent": False
            }
            return json.dumps(next_task)

    # If no pending tasks are found
    return json.dumps({"id": -1, "task_description": "No pending tasks found.", "requires_agent": False})

print("✅ Orchestration function restored.")

In [ ]:
import json
from typing import List, Dict, Union, Any



def update_add_task(description: str) -> str:
    """
    (UPDATED) Adds a new task to the list, automatically determining if it requires a specialist agent.
    This replaces the original 'add_task' function.
    """
    global _NEXT_ID
    
    # Logic to determine if a specialist agent is needed based on keywords
    # Added keywords to cover all 5 specialized agents
    requires_agent = any(k in description.lower() for k in ["book", "order", "shop", "ticket", "study", "plan", "quiz", "hotel", "flight", "course"])
    
    task_id = _NEXT_ID
    
    # Store the required flag in the global state dictionary
    _TASKS[task_id] = {
        "description": description, 
        "completed": False,
        "requires_agent": requires_agent 
    }
    _NEXT_ID += 1
    
    agent_status = "Requires Agent" if requires_agent else "Manual Task"
    return f"Task '{description}' successfully added with ID {task_id}. ({agent_status})"

# Overwrite the old function with the updated one
add_task = update_add_task

# --- NEW ORCHESTRATION TOOL ---

def process_next_task() -> str:
    """
    Identifies the next incomplete task, prioritizing Agent-assisted tasks first for delegation.
    This tool is called by the Master Orchestrator to trigger the sequential loop.
    Returns: A JSON string with the task details for the LLM to process.
    """
    next_task = None
    
    # 1. Prioritize Incomplete Agent Tasks
    for task_id, task in _TASKS.items():
        if not task["completed"] and task.get("requires_agent", False):
            next_task = {"id": task_id, **task}
            break
            
    # 2. Fallback to Incomplete Manual Tasks
    if next_task is None:
        for task_id, task in _TASKS.items():
            if not task["completed"] and not task.get("requires_agent", False):
                next_task = {"id": task_id, **task}
                break

    if next_task:
        return json.dumps({
            "next_task_id": next_task["id"],
            "task_description": next_task["description"],
            "requires_agent": next_task["requires_agent"]
        })
    else:
        return json.dumps({
            "next_task_id": None,
            "task_description": "All listed tasks are completed!",
            "requires_agent": False
        })
    
# Assign the new function for the orchestrator to use
orchestrate_task = process_next_task

print("✅ Task Management Logic Updated: 'add_task' now tracks agent requirement, and 'orchestrate_task' is ready for sequential processing.")

In [ ]:
# --- 1. The 'flights' Tool Function ---

# Import the Optional type
from typing import Optional 

def search_flights(
    origin: str, 
    destination: str, 
    departure_date: str, 
    # 💥 FIX IS HERE: Change str to Optional[str]
    return_date: Optional[str] = None, 
    # 💥 FIX IS HERE: Change float to Optional[float]
    max_price: Optional[float] = None
) -> str:
    """
    Searches for available flight options...
    """
    if not all([origin, destination, departure_date]):
        return "Error: Please specify the origin, destination, and departure date to search flights."
    
    # ... (rest of the function logic) ...
    # Simulating a successful tool response for agent testing
    price_info = f"Max Price: ${max_price}" if max_price else "No max price set"
    
    return (
        f"✅ Flight Query Successful: Searching from **{origin}** to **{destination}** on **{departure_date}**. "
        f"({price_info}). Found options starting at **$380** (Economy, BudgetLine) and **$750** (Premium, StarAir)."
    )

# You should also apply this to max_price and any other optional parameters in your tools!
# --- 2. The 'maps' Tool Function ---

def get_route_and_info(
    start_location: str, 
    end_location: str, 
    mode_of_transport: str = 'driving'
) -> str:
    """
    Calculates the route, distance, and estimated travel time between two points. 
    Use this for general routing, distances, or checking train/bus options.
    
    Args:
        start_location: The starting point (city, address, or station name).
        end_location: The ending point (city, address, or station name).
        mode_of_transport: The travel mode ('driving', 'walking', 'train', 'bus').
        
    Returns:
        A detailed summary of the route, distance, and time.
    """
    if not all([start_location, end_location]):
        return "Error: Both start and end locations are required for routing."
        
    # --- Simulated Mapping Service Response ---
    return (
        f"🚆 Route Calculated: From **{start_location}** to **{end_location}** via **{mode_of_transport}**. "
        f"Total Distance: 480 km. Estimated Travel Time: 3 hours 55 minutes. "
        "Train tickets are available every hour on the hour."
    )

In [ ]:
# Assigning the functions to the tool names used in the agent's instructions
flights = search_flights 
maps = get_route_and_info 

booking_agent = Agent(
    name="travel_booking_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    description="A specialized agent for searching and booking flight and train tickets.",
    instruction=(
        "You are a friendly and efficient travel assistant. "
        "Use the **flights** tool to search for flight options. "
        "Use the **maps** tool for train, bus, or general routing information. "
        "Always extract the origin, destination, and dates from the user's request before calling the tool."
    ),
    # Passing the callable functions as tools
    tools=[flights, maps], 
)

print("✅ Booking Agent defined with both flights and maps tools.")

In [ ]:
from typing import Optional
from google.adk.agents import Agent # Only if you didn't import this previously
from google.adk.tools import FunctionTool # Only if you didn't import this previously

# ----------------------------------------------------------------------
# 1. SHOPPING TOOL FUNCTIONS
# ----------------------------------------------------------------------

def search_product(
    query: str, 
    category: Optional[str] = None, 
    min_rating: Optional[float] = None
) -> str:
    """
    Searches online retailers for a specific product and returns key details and the best price found.
    
    Args:
        query: The name of the product to search for (e.g., "Sony noise-canceling headphones").
        category: Optional filter for the product category (e.g., "electronics").
        min_rating: Optional minimum customer rating (e.g., 4.5).

    Returns:
        A summary of the product, including specs and a simulated price.
    """
    if not query:
        return "Error: A search query is required to find a product."
    
    # --- Simulated Product Search Response ---
    details = f"Query: '{query}'. Category: {category if category else 'All'}. Min Rating: {min_rating if min_rating else 'None'}."
    
    return (
        f"✅ Product Found: We found the **{query}**. "
        f"Key Specs: 24-hour battery, Bluetooth 5.2, Available in Black/Silver. "
        f"The best price found is **₹19,999** (including taxes) at TechMart."
    )

def compare_prices(
    product_name: str, 
    retailer_list: Optional[list[str]] = None
) -> str:
    """
    Compares the price of a specific product across multiple online retailers.
    
    Args:
        product_name: The exact name of the product to compare.
        retailer_list: An optional list of retailers to check (e.g., ["Amazon", "Flipkart"]).

    Returns:
        A list of prices found at different retailers.
    """
    if not product_name:
        return "Error: The product name is required for price comparison."
    
    retailers = ", ".join(retailer_list) if retailer_list else "Amazon, Flipkart, TechMart"
    
    # --- Simulated Price Comparison Response ---
    return (
        f"📊 Price Comparison for **{product_name}** across {retailers}: \n"
        f"  - Amazon: ₹20,500 \n"
        f"  - Flipkart: ₹20,100 \n"
        f"  - TechMart (Recommended): **₹19,999** (Lowest Price) \n"
        "We recommend TechMart for the best value."
    )

In [ ]:

# Assign the functions to the tool names the LLM will call
product_search = search_product 
price_comparison = compare_prices 

# ----------------------------------------------------------------------
# 2. SHOPPING AGENT DEFINITION
# ----------------------------------------------------------------------

shopping_agent = Agent(
    name="online_shopping_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    description="A specialized agent for searching products, comparing prices, and finding online deals.",
    instruction=(
        "You are an expert online shopping assistant. "
        "Use the **product_search** tool to find product details and initial best prices. "
        "Use the **compare_prices** tool when the user asks to check prices at different retailers or compare options. "
        "Always extract the product name and any relevant details (category, budget) from the user's request before calling a tool."
    ),
    # Passing the callable functions as tools
    tools=[
        FunctionTool(product_search), 
        FunctionTool(price_comparison)
    ],
)
print("Shopping_agent created.")

In [ ]:
from typing import Optional, List



retry_config = {} 

# ----------------------------------------------------------------------
# 1. HOTEL BOOKING TOOL FUNCTIONS
# ----------------------------------------------------------------------

def search_hotels(
    location: str,
    check_in_date: str,
    check_out_date: str,
    min_rating: Optional[float] = 4.0,
    max_price_per_night: Optional[int] = None,
    amenities: Optional[List[str]] = None
) -> str:
    """
    Searches for available hotels in a specified location with filters for dates, price, and amenities.
    
    Args:
        location: The city or area where the user wants to stay (e.g., "Manhattan, New York").
        check_in_date: The date of arrival (YYYY-MM-DD format).
        check_out_date: The date of departure (YYYY-MM-DD format).
        min_rating: Minimum required guest rating (default is 4.0).
        max_price_per_night: Maximum price in local currency per night.
        amenities: A list of required amenities (e.g., ["pool", "free breakfast"]).

    Returns:
        A summary of the top hotel options found.
    """
    if not all([location, check_in_date, check_out_date]):
        return "Error: Location, check-in, and check-out dates are required to search for hotels."
    
    # --- Simulated Hotel Search Response ---
    amenity_list = ", ".join(amenities) if amenities else "No specific amenities requested"
    price_info = f"Max Price: {max_price_per_night}" if max_price_per_night else "No price limit"

    return (
        f"🏨 Search successful for {location} ({check_in_date} to {check_out_date}). "
        f"Filters: Min Rating {min_rating}, {price_info}. "
        f"Top 3 options found: \n"
        f"1. The Grand Residency (5-star, ₹8,500/night, has Pool) \n"
        f"2. City View Suites (4.5-star, ₹6,200/night, has Breakfast) \n"
        f"3. Budget Inn (4.0-star, ₹3,500/night)"
    )

def check_availability(
    hotel_name: str,
    check_in_date: str,
    check_out_date: str,
    room_type: str = "Standard King"
) -> str:
    """
    Checks the real-time availability and final booking price for a specific hotel and room type.
    
    Args:
        hotel_name: The specific name of the hotel.
        check_in_date: The date of arrival.
        check_out_date: The date of departure.
        room_type: The desired room type (e.g., "Deluxe Queen", "Suite").

    Returns:
        Confirmation of availability and the total price.
    """
    if not all([hotel_name, check_in_date, check_out_date]):
        return "Error: Hotel name and dates are required to check availability."
    
    # --- Simulated Availability Check Response ---
    total_nights = 3 # Simulated calculation

    return (
        f"✅ Availability Check: **{room_type}** at **{hotel_name}** is available! "
        f"Total Price for {total_nights} nights: **₹25,500**. "
        "Taxes and fees included. Ready to book."
    )

In [ ]:
# Assign the functions to the tool names the LLM will call
hotels_search = search_hotels
hotels_availability = check_availability

# ----------------------------------------------------------------------
# 2. HOTEL AGENT DEFINITION
# ----------------------------------------------------------------------

hotel_agent = Agent(
    name="hotel_reservation_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    description="A specialized agent for finding, filtering, and checking availability for hotel reservations.",
    instruction=(
        "You are a dedicated hotel booking specialist. "
        "Use the **hotels_search** tool to find general options based on location and dates. "
        "Use the **hotels_availability** tool to check detailed availability and final price for a specific hotel. "
        "Always extract the location, check-in, and check-out dates from the user's request."
    ),
    # Passing the callable functions as tools
    tools=[
        FunctionTool(hotels_search), 
        FunctionTool(hotels_availability)
    ],
)
print("hotel_agent is defined")

In [ ]:
from typing import Optional, List



retry_config = {} 

# ----------------------------------------------------------------------
# 1. MOVIE TICKET TOOL FUNCTIONS
# ----------------------------------------------------------------------

def search_showtimes(
    movie_name: str,
    city: str,
    date: str,
    time_preference: Optional[str] = None,
    theater_preference: Optional[str] = None
) -> str:
    """
    Searches for available showtimes for a specific movie in a given city and date.
    
    Args:
        movie_name: The title of the movie (e.g., "The Midnight Watchman").
        city: The city where the user wants to watch the movie (e.g., "Mumbai").
        date: The date for the show (YYYY-MM-DD format).
        time_preference: Optional time preference (e.g., "evening", "afternoon", "18:00").
        theater_preference: Optional preferred theater name.

    Returns:
        A formatted list of available showtimes at various theaters.
    """
    if not all([movie_name, city, date]):
        return "Error: Movie name, city, and date are required to search for showtimes."
    
    # --- Simulated Showtime Search Response ---
    time_filter = f" filtering for {time_preference}" if time_preference else ""
    theater_filter = f" at {theater_preference}" if theater_preference else ""

    return (
        f"🎬 Found showtimes for **{movie_name}** in {city} on {date}{time_filter}{theater_filter}:\n"
        f"1. PVR Phoenix: 14:30 (₹250), 18:00 (₹350), 21:15 (₹400)\n"
        f"2. Cinepolis Central: 15:15 (₹280), 19:45 (₹380)\n"
        f"3. INOX High Street: 16:00 (₹300), 22:00 (₹420)"
    )

def select_seats_and_book(
    movie_name: str,
    theater_name: str,
    showtime: str,
    number_of_tickets: int,
    seat_preferences: Optional[List[str]] = None
) -> str:
    """
    Simulates the final step: selecting seats, confirming availability, and completing the booking.
    
    Args:
        movie_name: The title of the movie.
        theater_name: The name of the theater.
        showtime: The selected showtime (e.g., "18:00").
        number_of_tickets: The quantity of tickets to book.
        seat_preferences: A list of desired seats (e.g., ["G10", "G11"]).

    Returns:
        A confirmation message with the total price.
    """
    if not all([movie_name, theater_name, showtime, number_of_tickets]):
        return "Error: Movie, theater, showtime, and ticket count are required for booking."
    
    # --- Simulated Booking Confirmation ---
    total_price = number_of_tickets * 350 # Example price
    seats = ", ".join(seat_preferences) if seat_preferences else "Auto-assigned seats"

    return (
        f"✅ Booking Confirmed! \n"
        f"Movie: **{movie_name}** ({showtime}) at {theater_name}. \n"
        f"Tickets: {number_of_tickets} ({seats}). \n"
        f"Total Price: **₹{total_price}**. Enjoy the show!"
    )

In [ ]:
# Assign the functions to the tool names the LLM will call
showtime_search = search_showtimes
booking_tickets = select_seats_and_book

# ----------------------------------------------------------------------
# 2. MOVIE AGENT DEFINITION
# ----------------------------------------------------------------------

movie_agent = Agent(
    name="movie_ticket_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    description="A specialized agent for searching showtimes and booking movie tickets.",
    instruction=(
        "You are an experienced movie ticket booking specialist. "
        "Use the **showtime_search** tool to find available movies, cities, dates, and times. "
        "Use the **booking_tickets** tool only when the user has provided the specific movie, theater, showtime, and number of tickets. "
        "Always extract the movie title, location, and date from the user's initial request."
    ),
    # Passing the callable functions as tools
    tools=[
        FunctionTool(showtime_search), 
        FunctionTool(booking_tickets)
    ],
)
print("movie_agent is defined")

In [ ]:
from typing import Optional, List


retry_config = {} 

# ----------------------------------------------------------------------
# 1. ACADEMIC STUDY TOOL FUNCTIONS (FIXED FOR IndexError)
# ----------------------------------------------------------------------

def generate_study_guide(
    topic: str,
    level: str,
    sections: Optional[List[str]] = None
) -> str:
    """
    Generates a structured study guide for a given topic, tailored to a specific academic level.
    
    Args:
        topic: The subject or concept to create a guide for (e.g., "Mitochondrial Function").
        level: The academic difficulty level (e.g., "High School", "Undergraduate", "Advanced").
        sections: Optional list of specific areas to focus on (e.g., ["Key Terms", "Processes", "Historical Context"]).

    Returns:
        A structured summary that serves as a study guide.
    """
    if not all([topic, level]):
        return "Error: Both the topic and academic level are required to generate a study guide."
    
    # --- Simulated Study Guide Response ---
    section_list = ", ".join(sections) if sections else "Key Concepts, Definitions, and Summary"

    return (
        f"📚 Study Guide Generated for **{topic}** (Level: {level}):\n"
        f"This guide focuses on the following areas: {section_list}.\n"
        f"\n---\n"
        f"**Key Concept 1: Structure and Purpose**\n"
        f"Mitochondria are the 'powerhouses' of the cell, generating most of the cell's supply of ATP through cellular respiration. They have a double membrane structure...\n"
        f"**Key Concept 2: The Krebs Cycle**\n"
        f"This occurs in the mitochondrial matrix and produces electron carriers (NADH and FADH2)...\n"
        f"**Summary:** Review the process of ATP generation and membrane structure."
    )

def create_quiz_questions(
    topic: str,
    num_questions: int,
    question_type: str = "Multiple Choice"
) -> str:
    """
    Creates a set of quiz questions to test understanding of a specific topic.
    
    Args:
        topic: The subject to create the quiz from.
        num_questions: The number of questions to generate.
        question_type: The format of the questions ("Multiple Choice", "True/False", "Short Answer").

    Returns:
        A formatted list of questions ready for the user to answer.
    """
    if not all([topic, num_questions]):
        return "Error: Topic and number of questions are required to create a quiz."
    
    # --- Simulated Quiz Response ---
    
    return (
        f"📝 Quiz Time! **{num_questions}** {question_type} questions on **{topic}**:\n"
        f"1. Which component of the cell is often referred to as the 'powerhouse'? (a) Nucleus (b) Ribosome (c) Mitochondria (d) Endoplasmic Reticulum\n"
        f"2. (True/False): The Citric Acid Cycle occurs in the inner mitochondrial membrane.\n"
        f"3. What is the primary molecule produced by mitochondria for energy transfer?\n"
        f"\n---\n"
        f"Good luck! Use the 'answer_quiz' tool if available, or ask me for the answers when finished."
    )

def create_study_plan(
    goal: str,
    duration_days: int,
    daily_study_hours: float,
    subjects: List[str]
) -> str:
    """
    Generates a high-level study timetable or plan based on goals and available time.
    
    Args:
        goal: The academic goal (e.g., "Pass Final Exam", "Complete Thesis Chapter", "Learn Python Basics").
        duration_days: The number of days until the deadline or for the study period.
        daily_study_hours: The average number of hours the user can commit per day.
        subjects: A list of subjects or topics to be covered (e.g., ["Calculus", "Physics", "English Literature"]).

    Returns:
        A structured weekly timetable/plan.
    """
    # --- ERROR HANDLING AND ROBUSTNESS FIX ---
    
    # 1. Fallback for subjects if the LLM fails to extract any
    if not subjects:
        inferred_subject = goal.replace(" final", "").strip().title()
        subjects = [inferred_subject if inferred_subject else "Study Topic"]
    
    if not all([goal, duration_days, daily_study_hours]):
        return "Error: Goal, duration, and study hours are all required to create a plan."

    # 2. FIX: Ensure the list is long enough for the simulated schedule (needs 3 unique indices)
    # This prevents the IndexError when the LLM only sends ['Python']
    while len(subjects) < 3:
        subjects.append(subjects[0])
        
    # --- Simulated Study Plan Response ---
    total_hours = duration_days * daily_study_hours
    # Use len(subjects) based on the *original* length for average calculation, or the fixed length for simplicity
    hours_per_subject = total_hours / len(subjects)
    
    # Now subjects[0], subjects[1], and subjects[2] are guaranteed to exist.
    plan_output = (
        f"📅 Study Plan Generated for Goal: **{goal}**\n"
        f"Duration: {duration_days} days. Total dedicated time: {total_hours:.1f} hours.\n"
        f"\n**Weekly Schedule (Simulated):**\n"
        f"| Day | Time Slot | Subject Focus |\n"
        f"| :--- | :--- | :--- |\n"
        f"| Mon | 18:00 - 20:00 | {subjects[0]} (Core Concepts) |\n"
        f"| Tue | 19:00 - 21:30 | {subjects[1]} (Practice Problems) |\n"
        f"| Wed | 17:00 - 19:00 | {subjects[2]} (Review & Quiz) |\n"
        f"| Thu | 18:00 - 20:30 | {subjects[0]} (Deep Dive) |\n"
        f"| Fri | 19:00 - 21:00 | {subjects[1]} (Review & Quiz) |\n"
        f"\n**Summary:** You have approximately {hours_per_subject:.1f} hours allocated per subject."
    )
    return plan_output





In [ ]:
# Assign the functions to the tool names the LLM will call
guide_generator = generate_study_guide
quiz_creator = create_quiz_questions
plan_creator = create_study_plan # NEW TOOL ASSIGNMENT


# ----------------------------------------------------------------------
# 2. STUDY AGENT DEFINITION (UPDATED)
# ----------------------------------------------------------------------

study_agent = Agent(
    name="academic_study_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    description="A specialized agent for creating study guides, generating quizzes, and assisting with academic learning and planning.",
    instruction=(
        "You are a friendly and structured academic tutor and planner. "
        "Use the **generate_study_guide** tool when the user needs content review. "
        "Use the **create_quiz_questions** tool when the user asks to test their knowledge. "
        "Use the **create_study_plan** tool when the user asks for a timetable, schedule, or study plan. "
        "Always extract the topic, academic level, and time constraints (duration, hours) from the user's request."
    ),
    # Passing the callable functions as tools
    tools=[
        FunctionTool(guide_generator), 
        FunctionTool(quiz_creator),
        FunctionTool(plan_creator) # ADDED NEW TOOL
    ],
)
print("study_agent is defined")

In [ ]:
import datetime # Need this if the get_current_date function is defined here or relies on it

# NOTE: Assuming you have delete_task, get_current_date, and all agents defined.
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.tools import FunctionTool, AgentTool
# Assuming SequentialAgent is used later for the root_agent
from google.adk.agents import SequentialAgent 

# --- MASTER TASK ORCHESTRATOR AGENT DEFINITION (REPLACES OLD CODE) ---

master_task_orchestrator = Agent(
    name="MasterTaskOrchestrator",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config # Assuming retry_config is defined
    ),
    description="The central agent that manages the user's to-do list, delegates complex tasks to specialist agents, and coordinates the sequential, interactive task completion workflow.",
    instruction=(
        "You are the central Task Manager, responsible for managing the to-do list and sequentially processing complex tasks."
        
        "\n**1. DATE/TIME AWARENESS (CRITICAL):**"
        "\n- **ALWAYS** call the **get_current_date()** tool immediately and **MANDATORILY** whenever the user mentions a relative date or time ('tomorrow,' 'next week,' etc.). Use the output to calculate the exact date before proceeding with delegation."
        "\n- **CRITICAL:** The output of get_current_date() is the absolute truth and overrides any internal knowledge of the date."

        "\n**2. IMMEDIATE TASK ACTION (Deletion and Viewing):**"
        "\n- **MANDATORY DELETION:** If the user asks to **REMOVE**, **DELETE**, **CLEAR**, or **GET RID OF** a task or completed item, you **MUST** immediately call **delete_task(ID)**. Do not offer alternative suggestions like 'view_list' or 'marking complete'; use the deletion tool."
        "\n- If the user asks to VIEW the list, immediately use the **view_list()** tool."

        "\n**3. Adding Tasks (Initial Prompt):**"
        "\n- When the user gives a list of things to do, use **add_task** for each item."

        "\n**4. Starting Workflow (Sequential Processing):**"
        "\n- When the user says 'Process the list' or 'Start my tasks', call **orchestrate_task**."

        "\n**5. Delegation and Interaction Loop (CRUCIAL):**"
        "\n- **If requires_agent is TRUE:** Delegate the task to the correct AgentTool, present the result, and then ask the confirmation question: 'Has this task (ID [ID] - [description]) been fully completed? Reply Yes or No?'"
        "\n- **If requires_agent is FALSE (Manual Task):** Ask the user: 'Task ID [ID] - [description] is a manual task. Have you completed this yet? Reply Yes or No?'"

        "\n**6. Marking Completion:**"
        "\n- If the user's next turn is 'Yes', immediately call **mark_complete(ID)** and then call **orchestrate_task** to start the next task."
        "\n- If the user's next turn is 'No', leave the task incomplete and call **orchestrate_task** to move to the next available task."
    ),
    tools=[
        # DATE AWARENESS 
        FunctionTool(get_current_date), 
        
        # Core To-Do Functions
        FunctionTool(add_task), 
        FunctionTool(view_list), 
        FunctionTool(mark_complete),
        FunctionTool(delete_task), 
        
        # New Orchestration Tool (This drives the sequential process)
        FunctionTool(orchestrate_task),
        
        # Specialized Delegation Agents
        AgentTool(booking_agent), 
        AgentTool(shopping_agent),
        AgentTool(hotel_agent),
        AgentTool(movie_agent),
        AgentTool(study_agent)
    ],
)

# For compatibility with your Cell (SequentialAgent definition)
todo_agent = master_task_orchestrator

print("✅ Master Orchestrator Agent successfully updated with reinforced deletion instructions.")

In [ ]:
# Assign the final root agent which is the orchestrator wrapped in a SequentialAgent
root_agent = SequentialAgent(
    name="RootTaskPipeline",
    description="The main application pipeline. Its only step is the MasterTaskOrchestrator, which handles all interactive tasks and delegation.",
    sub_agents=[
        master_task_orchestrator, 
    ],
)

print("✅ MasterTaskOrchestrator updated with real-time date awareness.")


In [ ]:
# Assuming your final agent is named 'root_todo_manager'
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner initialized with the Root Agent.")

In [ ]:
response_1 = await runner.run_debug(
    "I need to tackle five tasks for my busy week: First, I want to do Yoga Today. Second, I need to book a flight to London for December 15th. Third, can you check the showtimes and book two tickets for the new 'Star Wars' movie release next Friday? Fourth, I need a study guide on React Hooks for a quiz I have next week. Finally, please check hotel availability for a three-night stay near Hyde Park in London."
)
print(response_1)

In [ ]:
response_2 = await runner.run_debug(
    "today's date?please tell"
)
print(response_2)

In [ ]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

In [ ]:
url_prefix = get_adk_proxy_url()

In [ ]:
!adk web --url_prefix {url_prefix}